# Blockchain currencies 

### Building a blockchain coin form scratch in Python

It is recommended that the reader has read Blockchain fundamentals (if not already familiar with blockchain), as the basic structure is similar just with an appropriate vocabulary. Furthermore, there are a few principles worth explaining.

### Mining and proof of work:
Mining is mostly used as a way of introducing value into the system. By 'mining' a block the 'miner' can receive some amount of currency or reward and the proof of work legitimizes that action. Usually mining works by allocating a significant amount of computing power into making a block (here we simply represent that work with the time.sleep() function). As computers become more powerful the time for a task becomes less and thus it might be useful to increase the 'difficulty' of the work or reduce the amount of currency optained from mining a block.

### Transactions (peer to peer):
The third party skipping peer to peer transactions is probably one of the most well known aspects of blockchain. We thus introduce a Transaction class to simulate this. In reality most of these processes are much more complex, however, the following provides a decent overview of how blockchain coins work.

In [1]:
import time

In [2]:
class Event:  # The Block
    def __init__(self, transactions, prev_hash=None):
        self.transactions = transactions
        self.timestamp = time.time()
        self.prev_hash = prev_hash
        self.hash = self.hashing()
        
    def hashing(self):
        trade_hashes = []
        for transaction in self.transactions:
            trade_hashes.append(hash((transaction.from_id, transaction.to_id, transaction.amount)))
        return hash((tuple(trade_hashes), self.timestamp, self.prev_hash))
    
    def mine(self, difficulty):
        time.sleep(difficulty)
        print("Mining complete: {}".format(str(self)))
    

class Transaction:
    def __init__(self, from_id, to_id, amount):
        self.from_id = from_id
        self.to_id = to_id
        self.amount = amount

Now for the actual blockchain. It basically works as follows:

### Pending trades:
When a trade is made the transactions is added to a pending list awaiting the mining of a block. When a block is mined the pending transactions/trades is manifested within the block, thus validating them in the system. The miner is then paid in a new transaction, which is then added to the pending list and must await a new mining and so on.

<br>Another important principle to understand is balance:

### Balance:
With blockchain the idea of balance works differently than in traditional monetary systems. Instead of having an account with a balance, the balance of a profile or id in a blockchain is calculated by going through the chain itself (thus one can't manipulate one's balance).

In [3]:
class Coin:  # The BlockChain
    def __init__(self):
        self.history = [self.create_gen_event()]
        self.pending_trades = []
        
        self.mining_difficulty = 2
        self.mining_reward = 1000
        
        
    def create_gen_event(self):
        return Event([Transaction('Genesis Event', 'Genesis Event', 0)])
    
    def trade(self, from_id, to_id, amount):
        self.pending_trades.append(Transaction(from_id, to_id, amount))
    
    def mine_pending_trades(self, miner_id):  # add_block
        new_event = Event(self.pending_trades, self.history[-1].hash)
        new_event.mine(self.mining_difficulty)
        self.history.append(new_event)
        
        # self.pending_trades is reset with miner's reward as the first new pending trade
        self.pending_trades = [Transaction(None, miner_id, self.mining_reward)]
        
    def is_valid(self):
        for i in range(1, len(self.history)):
            prev = self.history[i-1]
            current = self.history[i]
            
            if current.hash != current.hashing():
                return False
            
            if current.prev_hash != prev.hashing():
                return False
        
        return True
    
    def get_balance(self, id_address):
        balance = 0
        
        for event in self.history:
            for transaction in event.transactions:
                if transaction.to_id == id_address:
                    balance += transaction.amount
                if transaction.from_id == id_address:
                    balance -= transaction.amount
        
        return balance
        
    
    def view_history(self):
        for event in self.history:
            print(vars(event))

Let's try to make a coin and do some trading:

In [4]:
random_coin = Coin()
random_coin.trade('person_1', 'person_2', 100)
random_coin.trade('person_1', 'person_3', 200)
print(random_coin.get_balance('person_2'))

0


Hmm, why does person_2 have a balance of 0? Because no mining has been perform and thus all current transactions are still pending. In order words the blockchain is still empty apart from the genesis block and an additional block is needed in order for the trades to actually become part of the chain. Let's try to mine a block then.

In [5]:
random_coin.mine_pending_trades('miner_1')
print(random_coin.get_balance('person_2'))
print(random_coin.get_balance('miner_1'))

Mining complete: <__main__.Event object at 0x00000217C08B7048>
100
0


Now that the transactions is validated person_2 has a balance of 100. The miner's balance meanwhile is still 0 because as before with the other transactions is now pending, waiting for the next block to be mined.

Next, let's have a look of our coins history:

In [6]:
random_coin.view_history()
random_coin.is_valid()

{'transactions': [<__main__.Transaction object at 0x00000217C08B7BA8>], 'timestamp': 1565205939.9254367, 'prev_hash': None, 'hash': 329821318631930327}
{'transactions': [<__main__.Transaction object at 0x00000217C08B79B0>, <__main__.Transaction object at 0x00000217C08B78D0>], 'timestamp': 1565205939.9434261, 'prev_hash': 329821318631930327, 'hash': -2297102039303025276}


True

As we can see we now have 2 blocks - our genesis block and our mined block with 2 transactions. We can also see that our chain is valid. Let's try tampering with our chain.

In [7]:
random_coin.history[1].transactions[0].amount = 1000000
print(random_coin.get_balance('person_2'))
random_coin.is_valid()

1000000


False

Well, we tried to make person_2 rich, but it didn't work.